In [1]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import numpy as np
import nibabel as nib
from scipy import stats

from mfs_tools.library.cifti_stuff import get_cortical_indices, get_subcortical_indices, get_cortical_data, get_axes_by_type, get_brain_model_axes


save_to = Path("/mnt/cache/pfm_python/")
reference_cifti_path = (
    save_to /
    "sub-ME01_task-rest_concatenated_and_demeaned_32k_fsLR.dtseries.nii"
)
distance_matrix_path = save_to / "dist_complete.npy"


In [2]:
# Load data
bold_cifti = nib.Cifti2Image.from_filename(reference_cifti_path)
distance_matrix = np.load(distance_matrix_path)


In [40]:
# Only calculate distance to real cortical vertices that may get used.
cort_idx = get_cortical_indices(bold_cifti)
subcort_idx = get_subcortical_indices(bold_cifti)
relevant_distances = distance_matrix[subcort_idx, :][:, cort_idx]


In [41]:
print(f"Filtered distance matrix down to {relevant_distances.shape[0]:,} "
      f"sub-cortical voxels by {relevant_distances.shape[1]:,} "
      f"cortical vertices")


Filtered distance matrix down to 25,647 sub-cortical voxels by 59,412 cortical vertices


In [80]:
# Determine which subcortical voxels are within 20mm of a cortical vertex.
smallest_distances = np.min(relevant_distances, axis=1)
outer_voxel_indices = np.where(smallest_distances <= 20)[0]


This generates a list of 19,137 voxels within 20mm of a cortical vertex.
This is exactly the same as the 19,137 voxels in Lynch's matlab code/data.
Next, we loop over each voxel near cortex, extract the BOLD from all
voxels within the 20mm threshold, and regress the cortical BOLD signal
from it.

In [78]:
def print_array_summary(a, desc):
    print(f"{desc} is shaped {a.shape}:")
    print("  [" + "".join(
        [", ".join([f"{v:0.2f}" for v in a[:5]])] +
        [", ..., "] +
        [", ".join([f"{v:0.2f}" for v in a[-5:]])]
    ) + "]")


In [75]:
i = 5

# Lynch's code for the first voxel results in [550 x 2560] BOLD, meaning
# that 550 voxels are within 20mm of the first voxel in our outer_voxel list.
# Our cifti data are transposed, [2560 x 550], but otherwise identical.
nearby_bold = bold_cifti.get_fdata()[:, distance_matrix[outer_voxel_indices[i], :] <= 20]

# Average the signal from all nearby voxels into a single time series
if nearby_bold.shape[1] > 1:
    nearby_bold = np.mean(nearby_bold, axis=1)



In [76]:
# We could use statsmodels or scikit-learn, but for a simple linear regression,
# we'll just stick with numpy.
# Regression outcome is this voxel's BOLD time series
voxel_index = subcort_idx[outer_voxel_indices[i]]
y = bold_cifti.get_fdata()[:, voxel_index]
# Regression data are the surrounding voxels' BOLD time series, with an intercept
# X = np.vstack([np.ones((1, len(nearby_bold))), nearby_bold.reshape(1, -1)])
X = nearby_bold

print(f"y is shaped {y.shape}; X is shaped {nearby_bold.shape}")

y is shaped (2560,); X is shaped (2560,)


In [79]:
# scipy's linregress function adds a column of ones internally,
# so we don't have to do it ourselves.
results = stats.linregress(nearby_bold, y)
predicted_y = results.intercept + results.slope * nearby_bold
residuals = y - predicted_y

print_array_summary(y, "Original BOLD")
print_array_summary(nearby_bold, "Regional average BOLD")
print_array_summary(residuals, "Residualized BOLD")


Original BOLD is shaped (2560,):
  [5.18, -36.65, -52.11, -5.04, -52.56, ..., 27.71, -10.79, -0.26, 45.62, 27.19]
Regional average BOLD is shaped (2560,):
  [-1.37, 9.72, 14.87, 19.99, 15.47, ..., -66.99, -69.48, -54.87, -28.57, -11.06]
Residualized BOLD is shaped (2560,):
  [4.98, -35.22, -49.92, -2.10, -50.28, ..., 17.85, -21.02, -8.34, 41.41, 25.56]


Above is an example of the voxel-wise regression processing. Below is exactly the same thing within a loop that will regress surrounding signal from each voxel near cortex.

In [81]:
adjusted_data = bold_cifti.get_fdata().copy()
for cifti_locus_index in outer_voxel_indices:
    # Extract all BOLD data within 20mm of this voxel
    nearby_bold = bold_cifti.get_fdata()[:, distance_matrix[cifti_locus_index, :] <= 20]
    if nearby_bold.shape[1] > 1:
        nearby_bold = np.mean(nearby_bold, axis=1)

    # Regress surrounding BOLD from this voxel's BOLD
    voxel_index = subcort_idx[cifti_locus_index]
    y = bold_cifti.get_fdata()[:, voxel_index]
    results = stats.linregress(nearby_bold, y)
    predicted_y = results.intercept + results.slope * nearby_bold
    residuals = y - predicted_y

    # Replace the BOLD data with residuals
    adjusted_data[:, voxel_index] = residuals

# We should now have a copy of the BOLD data,
# with each voxel near cortex cleaned of surrounding signal


In [82]:
# The adjusted data need to be packaged into a new Cifti2 file,
# just like the input file, and saved to disk.

adjusted_img = nib.Cifti2Image(
    adjusted_data, header=bold_cifti.header,
)
adjusted_img.to_filename(
    save_to /
    "sub-ME01_task-rest_concatenated_demeaned_and_regressed_32k_fsLR.dtseries.nii"
)
